In [ ]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical

from google.colab import drive
drive.mount('/content/drive')

print(os.listdir("/content/drive/MyDrive/Final Year Project - Lung Cancer Prediction/Datasets/CT Scan Images/"))

data_dir = Path("/content/drive/MyDrive/Final Year Project - Lung Cancer Prediction/Datasets/CT Scan Images/")
train_dir = data_dir / 'train'
test_dir = data_dir / 'test'

def load_train():
    cancer_dir = train_dir / 'Cancerous'
    non_cancer_dir = train_dir / 'Non_Cancerous'

    cancer_cases = cancer_dir.glob('*.jpg')
    non_cancer_cases = non_cancer_dir.glob('*.jpg')

    train_data = []
    train_label = []

    for img in cancer_cases:
        train_data.append(img)
        train_label.append('CANCER')

    for img in non_cancer_cases:
        train_data.append(img)
        train_label.append('NON_CANCER')

    df = pd.DataFrame(train_data)
    df.columns = ['images']
    df['labels'] = train_label
    df = df.sample(frac=1).reset_index(drop=True)
    return df

train_data = load_train()

def data_gen(data, batch_size):
    while True:
        batch_indices = np.random.choice(len(data), batch_size)
        batch_data = data.iloc[batch_indices]

        images = []
        labels = []
        for index, row in batch_data.iterrows():
            image_path = row['images']
            label = row['labels']

            img = cv2.imread(str(image_path))
            img = img_to_array(img)
            img = img / 255.0  # Normalizing

            images.append(img)
            labels.append(label)

        images = np.array(images)
        labels_one_hot = to_categorical(pd.Categorical(labels).codes, num_classes=2)

        yield images, labels_one_hot

batch_size = 16
nb_epochs = 5
train_gen = data_gen(data=train_data, batch_size=batch_size)
nb_train_steps = train_data.shape[0] // batch_size

model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))

model.add(Dense(2, activation='softmax'))  # Output layer with 2 units for 2 classes

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_gen,
    steps_per_epoch=nb_train_steps,
    epochs=nb_epochs
)

# After training, you can evaluate or predict using your test data similarly.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['test', 'train']
Epoch 1/5
14/20 [====================>.........] - ETA: 1:40 - loss: 105.1534 - accuracy: 0.7991

KeyboardInterrupt: ignored